In [ ]:
def directory(path):
    # Use list comprehension for efficiency
    image_path=[]
    for file in os.listdir(path):
        if file.endswith('.jpg'):
            img_path=os.path.join(path,file)
            image_path.append(img_path)
            image_path=image_path[:2000]
    
    print(f"Training samples: {len(image_path)}")
    #print(f"Test samples: {len(test_pathlist)}")
    
    return  image_path


In [ ]:
def preprocess_hr_to_lr(hr_path):
    # Read HR image
    img = tf.io.read_file(hr_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img,tf.float32)
    img = (img/127.5)-1
    

    # for low resolution images
    lr = tf.image.resize(img, [64, 64], method=tf.image.ResizeMethod.BILINEAR)
    noise = tf.random.normal(tf.shape(lr), mean=0.0,stddev=0.05)
    lr_img = tf.clip_by_value(lr+noise,-1,1)

    # for high resolution images
    hr_img = tf.image.resize(img, [256, 256], method=tf.image.ResizeMethod.BILINEAR)   
    return  hr_img,lr_img 

In [ ]:
def create_pipeline(hr_paths, batch_size=4, train_split=0.7, val_split=0.15, test_split=0.15):
    

    dataset = tf.data.Dataset.from_tensor_slices(hr_paths)
    
    
    dataset = dataset.shuffle(buffer_size=len(hr_paths), seed=42)
    
    # Calculate sizes for each split
    total_size = len(hr_paths)
    train_size = int(total_size * train_split)
    val_size = int(total_size * val_split)
    
    # Split the dataset
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size + val_size)
    
    
    def process_split(split):
        return (split
                .map(preprocess_hr_to_lr, num_parallel_calls=tf.data.AUTOTUNE)
                .batch(batch_size)
                .prefetch(tf.data.AUTOTUNE))
    
    
    train_dataset = process_split(train_dataset)
    val_dataset = process_split(val_dataset)
    test_dataset = process_split(test_dataset)
    
    return train_dataset, val_dataset, test_dataset


In [ ]:
train_dataset,val_dataset, test_dataset = create_pipeline(hr_list)
    